# RAG实践项目
## 一：理论学习

## 二：环境配置

## 三：读取汽车问答数据

读取问答数据集

In [1]:
import json
import pdfplumber

questions = json.load(open("questions.json", encoding='utf-8'))
questions[0:10]

[{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '', 'reference': ''},
 {'question': '"关于车辆的儿童安全座椅固定装置，在哪一页可以找到相关内容？"',
  'answer': '',
  'reference': ''},
 {'question': '“打开前机舱盖”的相关信息在第几页？', 'answer': '', 'reference': ''},
 {'question': '“打开前机舱盖”这个操作在哪一页？', 'answer': '', 'reference': ''},
 {'question': '“查看行车记录仪视频”这一项内容在第几页？', 'answer': '', 'reference': ''},
 {'question': '请问Lynk&Co领克汽车的事件数据记录系统（EDR）主要记录哪些信息？',
  'answer': '',
  'reference': ''},
 {'question': '问题：事件数据记录系统（EDR）中的数据是否可以被黑客利用进行恶意攻击？',
  'answer': '',
  'reference': ''},
 {'question': '问题：在国家环保法要求下，哪些情况下需要对车辆进行报废处理？', 'answer': '', 'reference': ''},
 {'question': '请问，如果车辆报废后，原车主是否还能使用该车辆的智能互联服务？',
  'answer': '',
  'reference': ''},
 {'question': '如何确保用车前的准备工作万无一失？', 'answer': '', 'reference': ''}]

读取可复制的pdf

读取读取第一页内容

In [2]:
pdf = pdfplumber.open("初赛训练数据集.pdf")
# len(pdf.pages) # 页数
pdf.pages[0].extract_text() # 读取第一页内容

'欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'

读取所有页内容

In [3]:
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })
pdf_content[0],pdf_content[10]

({'page': 'page_1',
  'content': '欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'},
 {'page': 'page_11',
  'content': '前言\n本手册相关的重要信息 为了凸显重点、便于阅读，中央显示屏中的插图突出显示重点部位，\n领克汽车销售有限公司（下称“Lynk&Co领克”）建议您在首次使用 其他部位做了一些处理，提示您不必关注这些部位。\n车辆前，认真阅读本手册内容。为了更好地理解本手册中的内容，您\n需要了解以下所有信息。 此图标表示“请勿这样做”或“请勿让此种情况发生”。\n提示信息\n警告！\n人身伤害 此图标表示“建议这样做”或“可以让此种情况发生”。\n■ 警告标识提醒您：如未按照该警告内容操作可能会对您或他人造\n成人身伤害或生命危险。\n注意！ 如果一幅插图显示多个零件，此图标表示零件位置。\n车辆损坏风险\n■ 注意标识提醒您：如未按照该注意事项操作可能会导致车辆损 如果一幅插图包含两个以上动作，此图标表示包含顺序的移动/\n坏。 动作。\n如果一幅插图只包含一个动作，此图标表示移动/动作。\n说明！\n帮助信息\n显示文本\n□ 在这里您可以找到一些帮助提示或有用的详细信息。\n车辆显示屏上显示的文本或消息的格式与普通文本不同（例如：取\n消、系统重置、路径编辑等），程序和指示分多个步骤显示（例如：\n车辆-车辆设置）。\n图标和插图说明\n敬告用户\n本手册中使用的插图和视频剪辑仅是示意图，展示车辆的某些特征或\n功能，用于辅助说明，以帮助您理解。此外车辆功能也会随着软件版\n为了您的安全，请您在驾驶车辆时遵守当地的法律法规，并遵循以下\n本升级或功能开通情况进行更迭，相应的内容及插图可能会和实车存\n注意事项：\n在差异，仅供参考，具体情况请以实车

# 五：语义检索

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('m3e-base')

question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

pdf_content_sentences[10]

D:\anaconda\envs\python39_nlp\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
D:\anaconda\envs\python39_nlp\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
D:\anaconda\envs\python39_nlp\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


'前言\n本手册相关的重要信息 为了凸显重点、便于阅读，中央显示屏中的插图突出显示重点部位，\n领克汽车销售有限公司（下称“Lynk&Co领克”）建议您在首次使用 其他部位做了一些处理，提示您不必关注这些部位。\n车辆前，认真阅读本手册内容。为了更好地理解本手册中的内容，您\n需要了解以下所有信息。 此图标表示“请勿这样做”或“请勿让此种情况发生”。\n提示信息\n警告！\n人身伤害 此图标表示“建议这样做”或“可以让此种情况发生”。\n■ 警告标识提醒您：如未按照该警告内容操作可能会对您或他人造\n成人身伤害或生命危险。\n注意！ 如果一幅插图显示多个零件，此图标表示零件位置。\n车辆损坏风险\n■ 注意标识提醒您：如未按照该注意事项操作可能会导致车辆损 如果一幅插图包含两个以上动作，此图标表示包含顺序的移动/\n坏。 动作。\n如果一幅插图只包含一个动作，此图标表示移动/动作。\n说明！\n帮助信息\n显示文本\n□ 在这里您可以找到一些帮助提示或有用的详细信息。\n车辆显示屏上显示的文本或消息的格式与普通文本不同（例如：取\n消、系统重置、路径编辑等），程序和指示分多个步骤显示（例如：\n车辆-车辆设置）。\n图标和插图说明\n敬告用户\n本手册中使用的插图和视频剪辑仅是示意图，展示车辆的某些特征或\n功能，用于辅助说明，以帮助您理解。此外车辆功能也会随着软件版\n为了您的安全，请您在驾驶车辆时遵守当地的法律法规，并遵循以下\n本升级或功能开通情况进行更迭，相应的内容及插图可能会和实车存\n注意事项：\n在差异，仅供参考，具体情况请以实车为准。\n11'

In [5]:
#  生成嵌入向量
question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)
    # normalize_embeddings=True：在生成嵌入时，将向量归一化处理（每个向量的L2范数变为1），确保相似度计算时的结果不会受到向量大小的影响。

for query_idx, feat in enumerate(question_embeddings):
    score = feat @ pdf_embeddings.T
    max_score_page_idx = score.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

with open('submit_m3e.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

In [6]:
from sentence_transformers import SentenceTransformer
from BCEmbedding import RerankerModel

model = SentenceTransformer("bce-embedding-base_v1")

question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)
    # normalize_embeddings=True：在生成嵌入时，将向量归一化处理（每个向量的L2范数变为1），确保相似度计算时的结果不会受到向量大小的影响。

for query_idx, feat in enumerate(question_embeddings):
    score = feat @ pdf_embeddings.T
    max_score_page_idx = score.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

with open('submit_bce.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

10/17/2024 14:47:55 - [INFO] -sentence_transformers.SentenceTransformer->>>    Use pytorch device_name: cuda
10/17/2024 14:47:55 - [INFO] -sentence_transformers.SentenceTransformer->>>    Load pretrained SentenceTransformer: bce-embedding-base_v1
D:\anaconda\envs\python39_nlp\lib\site-packages\transformers\modeling_utils.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We r